# Razão entre Idosas e Idosos por município do Brasil de 2010 a 2020

### Importações


In [4]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from geobr import read_state
from geobr import read_municipality
%matplotlib inline

In [5]:
from PIL import Image

## Obtendo informações geométricas dos municípios

In [6]:
# Gerando shape do brasil dividido em estados com a biblioteca GEOBR
shape_br = read_state(code_state='all')
# shape_br.plot()

shape_counties = read_municipality(code_muni='all', year=2018)
shape_counties.plot()
shape_counties

Exception: Some internal url is broken.Please report to https://github.com/ipeaGIT/geobr/issues

## Tratamento

In [ ]:
# Seleciona em 'pop_2010_2020.csv' apenas os atributos que vamos trabalhar
data = pd.read_csv('pop_2010_2020.csv')
data = data[['Ano','Sexo','Armenor', '60 a 65', '65 a 70', '70 a 75', '75 a 80', '80 a 85', '85 a 90', '90+', 'Total']]
data['60 a 65'] = data['60 a 65'].astype('int64')
data['65 a 70'] = data['65 a 70'].astype('int64')
data['70 a 75'] = data['70 a 75'].astype('int64')
data['75 a 80'] = data['75 a 80'].astype('int64')
data['80 a 85'] = data['80 a 85'].astype('int64')
data['85 a 90'] = data['85 a 90'].astype('int64')
data['90+'] = data['90+'].astype('int64')
data['Total'] = data['Total'].astype('int64')

# Somando o total de idosos por gênero por município e calculando a razão
data[["f", "m", "ratio"]] = ""
data['total_idosos'] = data['60 a 65'] + data['65 a 70'] + data['70 a 75'] + data['75 a 80'] + data['80 a 85'] + data['85 a 90'] + data['90+']

data_m = data.copy()
data_f = data.copy()
data_m = data_m.drop(data_m[data_m.Sexo == "f"].index)
data_f = data_f.drop(data_f[data_f.Sexo == "m"].index)

data_m.reset_index(inplace=True)
data_f.reset_index(inplace=True)

data

In [ ]:
data_f['ratio'] = data_f['total_idosos']/data_m['total_idosos']

data_f

### Agrupando dados geométricos e numéricos

In [ ]:
# Cria a coluna com o poligono representante de cada cidade e adiciona ela ao data_city
cities = shape_counties[['code_muni', 'geometry']]
data_f.reset_index(inplace=True)
data_city = data_f.merge(cities,how='inner', left_on='Armenor', right_on='code_muni')
data_city

## Plotando os mapas

In [ ]:
#Faz a plotagem dos mapas por semana epidemiológica 
for ano in range(2010,2021):
    ano_x = data_city[data_city.Ano == ano]
    
    #Fazendo o typecasting para GeoDataFrame
    ano_x = gpd.GeoDataFrame(ano_x)
    copy = gpd.GeoDataFrame(data_city)
    copy2 = shape_br
    copy2['coords'] = copy2['geometry'].apply(lambda x: x.representative_point().coords[:])
    copy2['coords'] = [coords[0] for coords in copy2['coords']]
    
    fig, ax = plt.subplots(figsize=(20, 15))
    ano_x.plot(column='ratio', cmap='bwr', vmin=data_city['ratio'].min(), legend=True,legend_kwds={'label': f"", 'orientation': "vertical"},vmax=data_city['ratio'].max(), ax=ax)
    
    ax.annotate(f'Ano {ano}', xy=(0.1,.225),xycoords='figure fraction',horizontalalignment='left', verticalalignment='top',fontsize=35)
    copy2.geometry.boundary.plot(color=None,edgecolor='#888888',linewidth = 0.3,ax=ax)
    ax.set_title(f"Razão entre idosas e idosos por município", fontdict={'fontsize':25})
    ax.axis('off')
    
    for i, j in copy2.iterrows():
        plt.annotate(s=j[1], xy=j['coords'], horizontalalignment='center', verticalalignment='center', fontsize=20)
    plt.show()
    # Salva cada mapa como um chart
    chart = ax.get_figure()
    chart.savefig(f'Ano_{ano}_IdosasPorIdosos.png', dpi=120)

# Agrupando as imagens no GIF

In [ ]:
#Cria um arquivo gif a partir de todos os pngs
images = []
for ano in range(2010, 2021):
    images.append(Image.open(f'Ano_{ano}_IdosasPorIdosos.png'))
    
images[0].save(f'Idosas x Idosos.gif', save_all=True, append_images=images, duration=500, loop=0)